## Run this notebook

You can launch this notbook using mybinder, by clicking the button below.

<a href="https://mybinder.org/v2/gh/NASA-IMPACT/veda-docs/HEAD?labpath=example-notebooks%2Fwfs.ipynb">
<img src="https://mybinder.org/badge_logo.svg" alt="Binder" title="A cute binder" width="150"/> 
</a>

## Approach

   1. Use `OWSLib` to determine what data is available and inspect the metadata
   2. Use `OWSLib` to filter and read the data
   3. Use `geopandas` and `folium` to analyze and plot the data

Note that the default examples environment is missing one requirement: `oswlib`. We can `pip install` that before we move on. 

In [1]:
!pip install OWSLib==0.28.1

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
from owslib.ogcapi.features import Features
import geopandas as gpd

## Look at the data that is availible through the OGC API

The datasets that are distributed throught the OGC API are organized into collections.
We can display the collections with the command: 

In [3]:
OGC_URL = "https://firenrt.delta-backend.com"

w = Features(url=OGC_URL)
w.feature_collections()

['public.eis_fire_newfirepix',
 'public.eis_fire_fireline',
 'public.eis_fire_perimeter',
 'public.st_subdivide',
 'public.st_hexagongrid',
 'public.st_squaregrid']

We will focus on the `public.eis_fire_fireline` collection and the `public.eis_fire_perimeter` collection. 

### Inspect the metatdata for public.eis_fire_perimeter collection

We can access information that drescribes the `public.eis_fire_perimeter`. 

In [4]:
perm = w.collection("public.eis_fire_perimeter")

We are particularly interested in the spatial and temporal extents of the data.

In [5]:
perm["extent"]

{'spatial': {'bbox': [[-124.5470962524414,
    24.073780059814453,
    -62.94002914428711,
    49.39276123046875]],
  'crs': 'http://www.opengis.net/def/crs/OGC/1.3/CRS84'},
 'temporal': {'interval': [['2023-03-07T00:00:00+00:00',
    '2023-03-27T00:00:00+00:00']],
  'trs': 'http://www.opengis.net/def/uom/ISO-8601/0/Gregorian'}}

In addition to getting metadata about the data we can access the queryable fields. Each of these fields will represent a column in our dataframe.

In [6]:
perm_q = w.collection_queryables('public.eis_fire_perimeter')
perm_q['properties']

{'wkb_geometry': {'$ref': 'https://geojson.org/schema/Geometry.json'},
 'ogc_fid': {'name': 'ogc_fid', 'type': 'number'},
 'n_pixels': {'name': 'n_pixels', 'type': 'number'},
 'n_newpixels': {'name': 'n_newpixels', 'type': 'number'},
 'farea': {'name': 'farea', 'type': 'number'},
 'fperim': {'name': 'fperim', 'type': 'number'},
 'flinelen': {'name': 'flinelen', 'type': 'number'},
 'duration': {'name': 'duration', 'type': 'number'},
 'pixden': {'name': 'pixden', 'type': 'number'},
 'meanfrp': {'name': 'meanfrp', 'type': 'number'},
 'isactive': {'name': 'isactive', 'type': 'number'},
 't': {'name': 't', 'type': 'string'},
 'fireid': {'name': 'fireid', 'type': 'number'}}

## Filter the data

It is always a good idea to do any data filtering as early as possible. In this example we know that we want the data for particular spatial and temporal extents. We can apply those and other filters using the `OWSLib` package.

In the below example we are:

 - choosing the `public.eis_fire_perimeter` collection 
 - subsetting it by space using the `bbox` parameter
 - subsetting it by time using the `datetime` parameter
 - filtering for fires over 5km^2 and over 2 days long using the `filter` parameter. The `filter` parameter lets us filter by the columns in 'public.eis_fire_perimeter' using SQL-style queries. 

NOTE: The `limit` parameter desginates the maximum number of objects the query will return. The default limit is 10, so if we want to all of the fire perimeters within certain conditions, we need to make sure that the limit is large.

In [7]:
perm_results = w.collection_items(
    'public.eis_fire_perimeter',   # name of the dataset we want
    bbox=['-106.8','24.5','-72.9','37.3'], # coodrinates of bounding box, 
    datetime=['2023-03-08T00:00:00+00:00/2023-03-20T12:00:00+00:00'], # date range
    limit=1000, # max number of items returned
    filter="farea>5 AND duration>2" # additional filters based on queryable fields 
)

The result is a dictionary containing all of the data and some summary fields. We can look at the keys to see what all is in there.

In [8]:
perm_results.keys()

dict_keys(['type', 'id', 'title', 'description', 'numberMatched', 'numberReturned', 'links', 'features'])

For instance you can check the total number of matched items and make sure that it is equal to the number of returned items. This is how you know that the `limit` you defined above is high enough.

In [9]:
perm_results['numberMatched'] == perm_results["numberReturned"]

True

You can also access the data directly in the browser or in an HTTP GET call using the constructed link.

In [10]:
perm_results['links'][1]['href']

'https://firenrt.delta-backend.com/collections/public.eis_fire_perimeter/items?bbox=-106.8%2C24.5%2C-72.9%2C37.3&datetime=2023-03-08T00%3A00%3A00%2B00%3A00%2F2023-03-20T12%3A00%3A00%2B00%3A00&limit=1000&filter=farea%3E5+AND+duration%3E2'

## Read data

In addition to all the summary fields, the `perm_results` dict contains all the data. We can pass the data into geopandas to make it easier to interact with.

In [11]:
df = gpd.GeoDataFrame.from_features(perm_results["features"])
df

geometry  ogc_fid  n_pixels  \
0   POLYGON ((-81.71187 29.10398, -81.71190 29.103...      186        25   
1   POLYGON ((-82.35959 27.21191, -82.35952 27.211...      377        44   
2   POLYGON ((-95.66867 30.56695, -95.66970 30.570...      620        79   
3   MULTIPOLYGON (((-84.37495 30.38736, -84.37498 ...      992        78   
4   POLYGON ((-84.68499 30.76543, -84.68501 30.765...     1140         5   
5   MULTIPOLYGON (((-84.62094 30.85949, -84.62425 ...     1145        89   
6   POLYGON ((-84.81950 31.49892, -84.82170 31.513...     1607        33   
7   MULTIPOLYGON (((-84.53935 31.37461, -84.53936 ...     1667        37   
8   POLYGON ((-84.05583 31.18532, -84.05584 31.185...     1727        21   
9   POLYGON ((-83.69479 31.72899, -83.69460 31.728...     1834        43   
10  MULTIPOLYGON (((-84.75087 32.51407, -84.75078 ...     1929       174   
11  POLYGON ((-84.53791 32.91535, -84.53791 32.915...     1973         4   
12  MULTIPOLYGON (((-83.71338 33.13964, -83.71327 ...     2052        82   
13  MULTIPOLYGON (((-85.33786 32.22803, -85.33767 ...     2392        18   
14  MULTIPOLYGON (((-85.51213 32.29998, -85.51196 ...     2402        62   
15  POLYGON ((-92.62616 31.75465, -92.62623 31.754...     3182       116   
16  POLYGON ((-94.22312 34.72191, -94.22292 34.721...     3452        37   
17  POLYGON ((-94.32359 35.74799, -94.32353 35.748...     3788        41   
18  POLYGON ((-90.95253 36.93086, -90.95244 36.930...     4176        95   
19  MULTIPOLYGON (((-80.03621 34.01988, -80.03622 ...     5282        29   
20  POLYGON ((-81.77293 33.29179, -81.77279 33.291...     5703        55   
21  MULTIPOLYGON (((-81.23368 31.45697, -81.23376 ...     6287        53   
22  POLYGON ((-100.38627 34.13080, -100.38776 34.1...     8836        40   
23  POLYGON ((-102.31280 31.63709, -102.31286 31.6...     9208         7   
24  POLYGON ((-98.35825 28.73597, -98.35823 28.735...     9710        69   
25  MULTIPOLYGON (((-97.43320 27.81192, -97.43321 ...     9803        71   

    n_newpixels      farea     fperim  flinelen  duration     pixden  meanfrp  \
0             0   7.734659  15.795540       0.0       7.0   3.232205      0.0   
1             0   5.748792   9.859231       0.0      11.0   7.653782      0.0   
2             0  15.029163  17.453145       0.0       3.0   5.256447      0.0   
3             0  18.333057  31.543427       0.0       4.0   4.254610      0.0   
4             0   5.540660   9.734886       0.0       5.0   0.902420      0.0   
5             0  19.624652  29.238119       0.0      10.5   4.535112      0.0   
6             0  11.261761  15.161253       0.0       5.0   2.930270      0.0   
7             0   7.459938  22.212764       0.0       3.0   4.959827      0.0   
8             0   7.858321  12.903165       0.0       2.5   2.672327      0.0   
9             0   5.259511   8.657170       0.0       4.0   8.175665      0.0   
10            0  31.477488  61.068995       0.0      10.0   5.527760      0.0   
11            0   6.314412  16.988807       0.0       4.0   0.633471      0.0   
12            0  16.625255  30.966800       0.0       8.0   4.932255      0.0   
13            0   5.037055  17.567075       0.0      11.0   3.573516      0.0   
14            0  11.029224  24.856197       0.0       4.0   5.621429      0.0   
15            0  21.505308  22.833460       0.0       5.0   5.394017      0.0   
16            0   7.661490  16.265716       0.0       5.0   4.829348      0.0   
17            0  10.302445  15.530567       0.0       4.0   3.979638      0.0   
18            0  15.173853  16.554001       0.0       6.0   6.260770      0.0   
19            0   6.117336  22.914300       0.0      16.0   4.740625      0.0   
20            0  10.289500  13.913957       0.0       3.0   5.345255      0.0   
21            0   8.192862  13.774701       0.0       8.0   6.469046      0.0   
22            0   8.485551  18.098234       0.0       6.0   4.713895      0.0   
23            0  14.036832  18.893386       0.0 

## Explore data

We can quickly explore the data by setting the coordinate reference system (`crs`) and using `.explore()`

In [12]:
df = df.set_crs("EPSG:4326")
df.explore()

## Visualize Most Recent Fire Perimeters with Firelines

If we wanted to combine collections to make more informative analyses, we can use some of the same principles. 

First we'll get the queryable fields, and the extents:

In [13]:
fline_q = w.collection_queryables('public.eis_fire_fireline')
fline_collection = w.collection("public.eis_fire_fireline")
fline_q['properties']

{'wkb_geometry': {'$ref': 'https://geojson.org/schema/Geometry.json'},
 'ogc_fid': {'name': 'ogc_fid', 'type': 'number'},
 'fireid': {'name': 'fireid', 'type': 'number'},
 't': {'name': 't', 'type': 'string'},
 'mergeid': {'name': 'mergeid', 'type': 'number'}}

### Read
Then we'll use those fields to get most recent fire perimeters and fire lines. Here we are plotting the most recent time with both fire line data and fire perimeter data. 

In [14]:
most_recent_time = min(max(*fline_collection['extent']['temporal']['interval']), max(*perm['extent']['temporal']['interval']))
print("Most Recent Time =", most_recent_time)

## Get the most recent fire perimeters
perm_results = w.collection_items(
    "public.eis_fire_perimeter",   
    datetime=most_recent_time,
    limit=1000,   
)
perimeters = gpd.GeoDataFrame.from_features(perm_results['features'])

## Get the most recent fire lines
perimeter_ids = perimeters.fireid.unique()
perimeter_ids = ",".join(map(str, perimeter_ids)) 

fline_results = w.collection_items(
    "public.eis_fire_fireline",
    limit=1000,
    filter="fireid IN (" + perimeter_ids + ")" # only the fires from the fire perimeter query above
    
)
fline = gpd.GeoDataFrame.from_features(fline_results['features'])

Most Recent Time = 2023-03-27T00:00:00+00:00


### Visualize

In [15]:
perimeters = perimeters.set_crs("epsg:4326")
fline = fline.set_crs("epsg:4326")

m = perimeters.explore()
m = fline.explore(m=m, color="orange")
m

## Download Data

Downloading pre-filtered data may be useful for working locally, or for working with the data in GIS software. 


We can download the dataframe we made by writing it out into a shapefile or into a GeoJSON file.  

    perimeters.to_file('perimeters.shp') 
    perimeters.to_file('perimeters.geojson', driver='GeoJSON')

## About the Data


The Fire data shown is generated by the FEDs algorithm. The FEDs algorithm tracks fire movement and severity by ingesting observations from the VIIRS thermal sensors on the Suomi NPP and NOAA-20 satellites. This algorithm uses raw VIIRS observations to generate a polygon of the fire, locations of the active fire line, and estimates of fire mean Fire Radiative Power (FRP). The VIIRS sensors overpass at ~1:30 AM and PM local time, and provide estimates of fire evolution ~ every 12 hours. The data produced by this algorithm describe where fires are in space and how fires evolve through time. This CONUS-wide implementation of the FEDs algorithm is based on [Chen et al 2020’s](https://www.nature.com/articles/s41597-022-01343-0) algorithm for California.


The data produced by this algorithm is considered experimental. 


## Collection Information

**public.eis_fire_perimeter**

Perimeter of cumulative fire-area.

| Column      | Description | Unit      |
| ----------- | ----------- | ----------- |
| meanfrp      | Mean fire radiative power. The weighted sum of the fire radiative power detected at each new pixel, divided by the number of pixels.   If no new pixels are detected, meanfrp is set to zero. | MW/(pixel area)  |
| t   | Time of detection.       | Datetime. yyyy-mm-ddThh:mm:ss. Local time.  |
| fireid   | Fire ID. Unique for each fire. Matches fireid. |Numeric ID |
| pixden   |Number of pixels divided by area of perimeter.   | pixels/Km^2|
| duration  |Number of days since first observation of fire. Fires with a single observation have a duration of zero. |Days|
| flinelen   | Length of active fire line, based on new pixels. If no new pixels are detected, flinelen is set to zero.   |Km |
| fperim   | Length of fire perimeter.        | Km   |
| farea   | Area within fire perimeter.        |Km^2    |
| n_newpixels   | Number of pixels newly detected since last overpass.      |pixels  |
| n_pixels   | Number of pixel-detections in history of fire. | pixels   |
| isactive   | Have new fire pixels been detected in the last 5 days?     | Boolean |
| ogc_fid   |The ID used by the OGC API to sort perimeters.       | Numeric ID   |
| geometry  |The shape of the perimeter.    | Geometry    |


**public.eis_fire_fireline**

Active fire line as estimated by new VIIRS detections.

**public.eis_fire_newfirepix**

New pixel detections that inform the most recent time-step’s perimeter and fireline calculation.

| Column      | Description | Unit      |
| ----------- | ----------- | ----------- |
| fireid   | ID of fire pixel associated with.      | Datetime. yyyy-mm-ddThh:mm:ss. Local time.   |
| t   | Time of detection.   | Title       |
| mergeid  | ID used to connect pixels to perimeters. Matches fireid      | Numeric ID      |
| ogc_fid   |The ID used by the OGC API to sort pixels.      | Numeric ID     |
